    - прогноз по максимальному количеству уроков в пиковый час нагрузки
    - прогноз по среднему кол-ву уроков в месяц

1. Посчитать медианное кол-во уроков за октябрь , ноябрь и декабрь 2022г. 
2. Найти пропорцию ноября и декабря от октября
3. Посчитать медианное число уроков за октбярь 2023 и умножить медианное зн. на пропорции из шага 1.

4. Сделать эти шаги для расчета пиковой нагрузки в час

In [5]:
import pandas as pd
import re
from colorama import Fore, Back, Style

from datetime import datetime
from sqlalchemy import create_engine


### Создадим движок для postgresql
engine = create_engine('postgresql://secret/secret')

**Выгрузка и обработка данных**

In [6]:
### Выгрузим данные из postgresql
query = '''select *
   from (
         SELECT *

         FROM (
               SELECT *,
                      ROW_NUMBER() OVER (PARTITION BY id ORDER BY modified_date DESC) AS rn
               FROM level_1_raw_data.lessons AS l

             ) drop_duplicates
         WHERE drop_duplicates.rn = 1) as l

where ended=TRUE AND regular_canceled IS NOT TRUE AND duration_min > 15  AND client_opozdal IS NOT TRUE;'''

lesson = pd.read_sql_query(query, engine)


# Создам колонку месяца, года, часа и даты
lesson['month'] = lesson['start_time'].dt.month
lesson['year'] = lesson['start_time'].dt.year
lesson['hour'] = lesson['start_time'].dt.hour
lesson['date'] = lesson['start_time'].dt.date


# Фильтрую месяцы в данных
oktober_22 = lesson.query('month == 10 & year == 2022')
oktober_23 = lesson.query('month == 10 & year == 2023')

november_22 = lesson.query('month ==11 & year == 2022')
december_22 = lesson.query('month == 12 & year == 2022')


---------

**Подсчет максимальной пик нагрузки в час и среднего кол-ва уроков**

In [17]:
lesson.query('month == 9 & year == 2023')



,duration_min,ended,start_time,student,created_date,created_by,modified_date,regular,uid,end_time,...,vznaniya_link_teacher,vznaniya_link_student,additional_id,admin_cancelled,amo_lead_id,rn,month,year,hour,date
2985,60.0,True,2023-09-24 13:00:00,None,2021-11-04 16:00:07.739,admin_user_tutgood_live,2023-09-24 14:02:26.767,False,4265.0,2023-09-24 14:00:00,...,https://webinar.tutgood.online/b/?group=226974...,https://webinar.tutgood.online/b/dtc-ved-siu-bdy,NaN,None,NaN,1,9.0,2023.0,13.0,2023-09-24
5936,40.0,True,2023-09-20 13:00:00,None,2021-12-11 10:44:37.514,admin_user_tutgood_live,2023-09-20 13:45:43.738,False,12435.0,2023-09-20 14:00:00,...,https://webinar.tutgood.online/b/?group=222305...,https://webinar.tutgood.online/b/nuq-kxm-axj-omv,NaN,None,NaN,1,9.0,2023.0,13.0,2023-09-20
10774,60.0,True,2023-09-09 16:00:00,None,2022-01-27 17:46:16.333,admin_user_tutgood_live,2023-09-03 10:02:55.329,False,23123.0,2023-09-09 17:00:00,...,None,None,2.0,True,NaN,1,9.0,2023.0,16.0,2023-09-09
31568,60.0,True,2023-09-18 09:00:00,None,2022-04-16 11:02:04.115,admin_user_tutgood_live,2023-09-18 09:57:51.018,False,59061.0,2023-09-18 10:00:00,...,https://webinar.tutgood.online/b/?group=219204...,https://webinar.tutgood.online/b/qvt-apk-jpt-h2p,NaN,None,NaN,1,9.0,2023.0,9.0,2023-09-18
56780,37.0,True,2023-09-28 16:00:00,None,2022-07-23 12:12:11.168,admin_user_tutgood_live,2023-09-28 18:55:13.947,False,97830.0,2023-09-28 17:00:00,...,https://webinar.tutgood.online/b/?group=232747...,https://webinar.tutgood.online/b/mar-e5x-3kl-2ik,NaN,None,30292017.0,1,9.0,2023.0,16.0,2023-09-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435334,60.0,True,2023-09-30 17:00:00,1678545436194x555858833403510800,2023-09-30 15:15:34.086,1678545436194x555858833403510800,2023-10-02 20:38:54.837,True,651011.0,2023-09-30 18:00:00,...,https://webinar.tutgood.online/b/?group=235087...,None,0.0,None,NaN,1,9.0,2023.0,17.0,2023-09-30
435377,51.0,True,2023-09-30 17:00:00,1674404647196x922538923933445900,2023-09-30 16:06:16.278,1674404647196x922538923933445900,2023-10-02 13:07:07.685,False,651074.0,2023-09-30 18:00:00,...,https://webinar.tutgood.online/b/?group=235084...,None,0.0,None,NaN,1,9.0,2023.0,17.0,2023-09-30
435378,35.0,True,2023-09-30 17:00:00,1662201916578x493202747430479000,2023-09-30 16:14:46.999,1662201916578x493202747430479000,2023-10-02 15:17:57.909,False,651078.0,2023-09-30 18:00:00,...,https://webinar.tutgood.online/b/?group=235093...,None,0.0,None,NaN,1,9.0,2023.0,17.0,2023-09-30
435380,45.0,True,2023-09-30 16:00:00,1680111420943x970202448415340500,2023-09-30 16:18:27.349,1680111420943x970202448415340500,2023-09-30 21:57:23.543,False,651080.0,NaT,...,https://webinar.tutgood.online/b/?group=235059...,None,0.0,None,NaN,1,9.0,2023.0,16.0,2023-09-30


In [7]:
# Функция по подсчету максимального числа уроков в час по месяцам
def max_lesson_per_hour(data):
    data = data.groupby(['date', 'hour']).size().reset_index(name='Lesson_Count')\
               .sort_values('Lesson_Count', ascending = False)
    
    max_data = data['Lesson_Count'].max()
    
    return max_data


# Переменные с макс пиковым числом уроков в час по месяцу
max_oktober_22 = max_lesson_per_hour(oktober_22)
max_november_22 = max_lesson_per_hour(november_22)
max_december_22 = max_lesson_per_hour(december_22)

max_oktober_23 = max_lesson_per_hour(oktober_23)


print(Fore.BLACK + f'в Октябре в пик нагрузки макс. кол-во уроков в час было {max_oktober_22}')
print(Fore.BLACK + f'в Ноябре в пик нагрузки макс. кол-во уроков в час было {max_november_22}')
print(Fore.BLACK + f'в Декабре в пик нагрузки макс. кол-во уроков в час было {max_december_22}')

print(Fore.BLACK + f'в Октябре 2023 в пик нагрузки макс. кол-во уроков в час было {max_oktober_23}')

в Октябре в пик нагрузки макс. кол-во уроков в час было 125
в Ноябре в пик нагрузки макс. кол-во уроков в час было 179
в Декабре в пик нагрузки макс. кол-во уроков в час было 193
в Октябре 2023 в пик нагрузки макс. кол-во уроков в час было 194


In [8]:
# Cреднее количество уроков в месяц 
coutn_okt_22 = oktober_22.groupby('date').agg({'id':'count'}).reset_index()['id'].median()
coutn_nov_22 = november_22.groupby('date').agg({'id':'count'}).reset_index()['id'].median()
coutn_dec_22 = december_22.groupby('date').agg({'id':'count'}).reset_index()['id'].median()
coutn_okt_23 = oktober_23.groupby('date').agg({'id':'count'}).reset_index()['id'].median()


print(Fore.BLACK + f'в Октябре медианное кол-во уроков было {round(coutn_okt_22)}')
print(Fore.BLACK + f'в Ноябре медианное кол-во уроков было {round(coutn_nov_22)}')
print(Fore.BLACK + f'в Декабре медианное кол-во уроков было {round(coutn_dec_22)}')

print(Fore.BLACK + f'в Октябре 2023 медианное кол-во уроков было {round(coutn_okt_23)}')

в Октябре медианное кол-во уроков было 708
в Ноябре медианное кол-во уроков было 940
в Декабре медианное кол-во уроков было 1003
в Октябре 2023 медианное кол-во уроков было 1026


In [13]:
# Cреднее количество уроков в месяц 
shape_okt_22 = oktober_22.shape[0]
shape_nov_22 = november_22.shape[0]
shape_dec_22 = december_22.shape[0]
shape_okt_23 = oktober_23.shape[0]


print(Fore.BLACK + f'в Октябре кол-во уроков было {round(shape_okt_22)}')
print(Fore.BLACK + f'в Ноябре кол-во уроков было {round(shape_nov_22)}')
print(Fore.BLACK + f'в Декабре кол-во уроков было {round(shape_dec_22)}')

print(Fore.BLACK + f'в Октябре 2023 медианное кол-во уроков было {round(shape_okt_23)}')

в Октябре кол-во уроков было 21534
в Ноябре кол-во уроков было 27395
в Декабре кол-во уроков было 30750
в Октябре 2023 медианное кол-во уроков было 28563


**Поиск пропорций**

In [14]:
# Функция для вычисления пропорции между октябрем и другими месяцами
def proportion_between_oktober(data, oktober):
    prop = round(data / oktober , 2)
    
    return prop

    Прогноз максимальной нагрузки в час 

In [15]:
# Пременные с пропорциями
okt_nov_prop = proportion_between_oktober(max_november_22, max_oktober_22)
okt_dec_prop = proportion_between_oktober(max_december_22, max_oktober_22)

# Прогнозное количество макс нагрузки в час в ноябре и декабре 2023
prognoz_v_chas_november = max_oktober_23 * okt_nov_prop
prognoz_v_chas_december = max_oktober_23 * okt_dec_prop

    Прогноз среднего количества уроков

In [22]:
# Пременные с пропорциями
okt_nov_prop_count = proportion_between_oktober(shape_nov_22, shape_okt_22)
okt_dec_prop_count = proportion_between_oktober(shape_dec_22, shape_okt_22)

# Прогнозное среднее количество уроков в ноябре и декабре 2023
prognoz_srednee_kolichestvo_november = shape_okt_23 * okt_nov_prop_count
prognoz_srednee_kolichestvo_december = shape_okt_23 * okt_dec_prop_count

In [25]:
print(Fore.BLACK + f'Прогнозируемая нагрузка макс. кол-во уроков в час в Ноябре 2023 = {round(prognoz_v_chas_november)}')
print(Fore.BLACK + f'Прогнозируемая нагрузка макс. кол-во уроков в час в Декабре 2023 = {round(prognoz_v_chas_december)}')
print(Fore.BLACK + f'Прогнозируемое среднее количество уроков в Ноябре 2023 = {round(prognoz_srednee_kolichestvo_november):,.0f}')
print(Fore.BLACK + f'Прогнозируемое среднее количество уроков в Декабре 2023 = {round(prognoz_srednee_kolichestvo_december):,.0f}')


Прогнозируемая нагрузка макс. кол-во уроков в час в Ноябре 2023 = 277
Прогнозируемая нагрузка макс. кол-во уроков в час в Декабре 2023 = 299
Прогнозируемое среднее количество уроков в Ноябре 2023 = 36,275
Прогнозируемое среднее количество уроков в Декабре 2023 = 40,845


------